In [117]:
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import datetime
from datetime import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import scipy
from scipy.stats import reciprocal
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (Input, Convolution1D, Dense, MaxPooling1D,MaxPooling2D,AveragePooling2D,
                                    Flatten, Dropout, Activation, average,
                                    BatchNormalization, Reshape, Conv2D, ActivityRegularization)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
from keras import backend as K


def enc_dec(optimizer, lr, decay):
    
    init = 'he_normal'
    
    stacked_enconder = Sequential([
                                   Dense(276, input_dim=276, kernel_initializer=init,activation='linear'),
                                   Dense(250, kernel_initializer=init, activation='elu'),
                                   Dense(225, kernel_initializer=init, activation='elu'),
                                   Dense(200, kernel_initializer=init, activation='elu'),
                                   Dense(150, kernel_initializer=init, activation='elu'),
                                   Dense(125, kernel_initializer=init, activation='elu'),
                                   Dense(100, kernel_initializer=init, activation='elu'),
                                   Dense(50, kernel_initializer=init, activation='elu')
                                   #ActivityRegularization(l1=1e-4)
                                  ])

    stacked_decoder  = Sequential([
                                   Dense(50, kernel_initializer=init, activation='elu', input_dim=50),
                                   Dense(100, kernel_initializer=init, activation='elu'),
                                   Dense(125, kernel_initializer=init, activation='elu'),
                                   Dense(150, kernel_initializer=init, activation='elu'),
                                   Dense(200, kernel_initializer=init, activation='elu'),
                                   Dense(225, kernel_initializer=init, activation='elu'),
                                   Dense(250, kernel_initializer=init, activation='elu'),
                                   Dense(276, input_dim=276, kernel_initializer=init ,activation='linear')
                                  ])
    
    
    stacked_ae = Sequential([stacked_enconder, stacked_decoder])
    opt = Adam(lr=lr ,decay=decay)
    stacked_ae.compile(loss='mse', optimizer=opt)
    
    return stacked_ae, stacked_enconder, stacked_decoder

In [118]:
#st_ae, en, dec  = enc_dec('Adam', 0.0006, 0.001)
st_ae, en, dec  = enc_dec('Adam', 0.001, 0.001)

In [ ]:
my_callbacks =  [keras.callbacks.TerminateOnNaN() ] #keras.callbacks.EarlyStopping(patience=100), 
#https://medium.com/@am.benatmane/keras-hyperparameter-tuning-using-sklearn-pipelines-grid-search-with-cross-validation-ccfc74b0ce9f

history = st_ae.fit(X_training, X_training, callbacks= my_callbacks,validation_data=(X_test, X_test), epochs=2500,  batch_size=25, verbose=2 ) 

Epoch 1/2500
40/40 - 1s - loss: 57.4876 - val_loss: 12.2740
Epoch 2/2500
40/40 - 1s - loss: 8.5390 - val_loss: 6.4359
Epoch 3/2500
40/40 - 1s - loss: 6.1748 - val_loss: 5.8500
Epoch 4/2500
40/40 - 1s - loss: 6.0202 - val_loss: 5.6860
Epoch 5/2500
40/40 - 1s - loss: 5.4308 - val_loss: 5.0813
Epoch 6/2500
40/40 - 1s - loss: 5.0372 - val_loss: 4.8162
Epoch 7/2500
40/40 - 1s - loss: 4.7756 - val_loss: 4.5683
Epoch 8/2500
40/40 - 1s - loss: 4.5536 - val_loss: 4.4270
Epoch 9/2500
40/40 - 1s - loss: 4.4995 - val_loss: 4.4204
Epoch 10/2500
40/40 - 1s - loss: 4.2774 - val_loss: 4.1053
Epoch 11/2500
40/40 - 1s - loss: 4.1358 - val_loss: 4.1031
Epoch 12/2500
40/40 - 1s - loss: 4.0973 - val_loss: 4.0624
Epoch 13/2500
40/40 - 1s - loss: 4.0027 - val_loss: 3.8385
Epoch 14/2500
40/40 - 1s - loss: 3.8370 - val_loss: 3.7079
Epoch 15/2500
40/40 - 1s - loss: 3.6129 - val_loss: 3.5150
Epoch 16/2500
40/40 - 1s - loss: 3.4526 - val_loss: 3.4341
Epoch 17/2500
40/40 - 1s - loss: 3.4103 - val_loss: 3.3287
Epoc

In [91]:
X_training_enc = np.array(en(X_training))
X_test_enc     = np.array(en(X_test))

In [116]:
if __name__ == "__main__":

    # For every compound generate a coulomb matrix
    for mol in compounds:

        mol.generate_coulomb_matrix(size=23, sorting="row-norm")
        # mol.generate_bob(size=23, asize={"O":3, "C":7, "N":3, "H":16, "S":1})
            

    # Make a big 2D array with all the 
    X = np.array([mol.representation for mol in compounds], dtype=np.float32)
    energy_pbe0 = np.array(energy_pbe0,  dtype=np.float32)
    # X = np.array([mol.bob for mol in compounds])

    print(energy_pbe0)

    # Assign 1000 first molecules to the training set
    X_training = X[:1000]
    Y_training = energy_pbe0[:1000]

    # Y_training = energy_delta[:1000]

    # Assign 1000 first molecules to the training set
    X_test = X[-1000:]
    Y_test = energy_pbe0[-1000:]
    # Y_test = energy_delta[-1000:]
   
    # Calculate the Gaussian kernel
    sigma = 2000
    K = gaussian_kernel(X_training_enc, X_training_enc, sigma)
    print(K)

    # Add a small lambda to the diagonal of the kernel matrix
    K[np.diag_indices_from(K)] += 1e-8

    # Use the built-in Cholesky-decomposition to solve
    alpha = cho_solve(K, Y_training) 

    #print(alpha)

    # Assign 1000 last molecules to the test set
    X_test = X[-1000:]
    Y_test = energy_pbe0[-1000:]

    # calculate a kernel matrix between test and training data, using the same sigma
    Ks = gaussian_kernel(X_test_enc, X_training_enc, sigma)

    # Make the predictions
    Y_predicted = np.dot(Ks, alpha)

    # Calculate mean-absolute-error (MAE):
    print(np.mean(np.abs(Y_predicted - Y_test)))

[-1633.28 -1212.67 -1531.73 ... -1728.7  -1506.76 -1866.88]
[[1.         0.99953743 0.99965382 ... 0.99981073 0.99967342 0.9995744 ]
 [0.99953743 1.         0.9997474  ... 0.99973735 0.99994014 0.99967463]
 [0.99965382 0.9997474  1.         ... 0.99982025 0.99973185 0.99966024]
 ...
 [0.99981073 0.99973735 0.99982025 ... 1.         0.9998112  0.99983571]
 [0.99967342 0.99994014 0.99973185 ... 0.9998112  1.         0.99973056]
 [0.9995744  0.99967463 0.99966024 ... 0.99983571 0.99973056 1.        ]]
26.764004133105278
